# Introduction

## Next Steps

URL Of the OG List: https://docs.google.com/spreadsheets/d/1Mch2d3KO2Hc1n4KBpHbNVPtqnWBQ5RX6KeRCq4eOKac/edit#gid=1499993616

## The Beautiful For Loop I have too much Pride in:

```python

for row in cube_slice.iterrows():
        if rq.get(cube_slice['URL'][index]).ok == True:
            df = pd.json_normalize(rq.get(cube_slice['URL'][index]).json().get('cards').get('mainboard'), sep='_')
            df['ID'] = cube_slice['ID'][index]
            df['Cube Name'] = cube_slice['Cube Name'][index]
            df['Cubecon Type'] = 'Pre-Accepted'
            df_list.append(df)
            index+=1
        else:
            print('Error Encountered At Index:', index)
            index+=1
```

This is where the magic happens.  

We start by iterating for each *row* in our Cube Export.  We want each URL that we've created to generate a JSON response _only for that row_.  This is where we call our `[index]` value because you need to capture the specific row related to each field.  

However, before we begin, we don't want our Loop to get broken by a failed HTTP Request for information, so we call the `req.get().ok` parameter to ensure that we _start_ with a `200` response aka `True` before beginning.

We start by calling our URL related to our Index, and returning the HTTP Response of the get into JSON for parsing.  This mostly works because of the `http://cubecobra.com/cube/api/cubeJSON/` API call that returns the _entire_ webpage into JSON format.  Of the response, we're only interested in the `{cards:}` portion, and of those, the `{'mainboard':}` for analysis puroses.  However, because the response is in nested JSON, we need to flatten it because there exist really important attributes in the subsection of `{details:}` for future analysis.  Then the object is turned into a Pandas DataFrame we call "df".

However, if we just stopped there, we would lose a level of attribution when all of the DataFrames are joined together.  Currently the Cube-Level information is outside of the `{mainboard:}` attribute so we instead continue to leverage the `[index]` parameter by adding the "Cube Name" and "Cube ID" to the DataFrame, as well as noting the Type of Cube it is within the larger voting sphere of cubes.

Then the DataFrame is appended to our "df_list" and the index is increased for the next row.  If our response initially encountered a bad HTTP Request, then we print an Error Message and increase the Index so we can continue.


In [1]:
import pandas as pd
import numpy as np
import requests as rq

In [2]:
# Import The Cube List
cube_export = pd.read_csv('/Users/tpug/Jupyter Work/Magic The Gathering Projects/Cubecon_2023/Cubecon-2023/Data Files/Copy of CubeCon 2023 Public Voting Dates - Pre-Accepted.csv')

In [3]:
cube_export.head()

,Cube Name and Link,Discord,Twitter Link,Twitter,Pronouns,Cube Name,URL
0,Regular Cube,Anthony#1172,ahmattox,ahmattox,"he/him, they/them",Regular Cube,https://cubecobra.com/cube/overview/regular
1,Turbo Cube,Anthony#1172,ahmattox,ahmattox,"he/him, they/them",Turbo Cube,https://cubecobra.com/cube/overview/turbo
2,Creatureless Cube,beeks#2291,rolltotweet,rolltotweet,they/them,Creatureless Cube,https://cubecobra.com/cube/overview/no-creatures
3,Dragons of Winter's Night,Chill MtG#5975,ChillMtGTV,ChillMtGTV,he/him,Dragons of Winter's Night,https://cubecobra.com/cube/overview/down
4,Eleusis,CulticCube#6756,CulticCube,CulticCube,he/him,Eleusis,https://cubecobra.com/cube/overview/culticcube


In [4]:
# Split URL Into ID, Expand into Fields
# From there, Give me only the "5th" column
# Add to DataFrame as the "ID" Field
cube_export['ID']=cube_export['URL'].str.split('/',expand=True,n=5).loc[:,5]
cube_export.head(30)

,Cube Name and Link,Discord,Twitter Link,Twitter,Pronouns,Cube Name,URL,ID
0,Regular Cube,Anthony#1172,ahmattox,ahmattox,"he/him, they/them",Regular Cube,https://cubecobra.com/cube/overview/regular,regular
1,Turbo Cube,Anthony#1172,ahmattox,ahmattox,"he/him, they/them",Turbo Cube,https://cubecobra.com/cube/overview/turbo,turbo
2,Creatureless Cube,beeks#2291,rolltotweet,rolltotweet,they/them,Creatureless Cube,https://cubecobra.com/cube/overview/no-creatures,no-creatures
3,Dragons of Winter's Night,Chill MtG#5975,ChillMtGTV,ChillMtGTV,he/him,Dragons of Winter's Night,https://cubecobra.com/cube/overview/down,down
4,Eleusis,CulticCube#6756,CulticCube,CulticCube,he/him,Eleusis,https://cubecobra.com/cube/overview/culticcube,culticcube
5,Petty Nobility,CulticCube#6756,CulticCube,CulticCube,he/him,Petty Nobility,https://cubecobra.com/cube/overview/pettynobility,pettynobility
6,Dekkaru Cube,DEKKARU#2784,GwenDekker,GwenDekker,he/him,Dekkaru Cube,https://cubecobra.com/c/dekkaru,None
7,Tiny Leaders,DEKKARU#2784,GwenDekker,GwenDekker,he/him,Tiny Leaders,https://cubecobra.com/cube/overview/tinyleaders,tinyleaders
8,Emma Partlow's Peasant Cube,EmmaPartlow#0071,emmadpartlow,emmadpartlow,"she/her, they/them",Emma Partlow's Peasant Cube,https://cubecobra.com/cube/list/emma-peasant-c...,emma-peasant-cube-2022
9,Old Border Foil Cube,GLHF Games#0204,glhf_games,glhf_games,he/him,Old Border Foil Cube,https://cubecobra.com/cube/list/obf,obf


In [5]:
cube_slice = cube_export.loc[:, ['Cube Name', 'ID']]
# Manually Replacing wrong IDs for the Iteration
cube_slice.loc[6, ['ID']] = ['dekkaru']
cube_slice.loc[26, ['ID']] = ['UBHC']
cube_slice.loc[8, ['ID']] = ['emma-peasant-cube-2023']

cube_slice['URL'] = 'http://cubecobra.com/cube/api/cubeJSON/' + cube_slice['ID']
cube_slice['URL']

0        http://cubecobra.com/cube/api/cubeJSON/regular
1          http://cubecobra.com/cube/api/cubeJSON/turbo
2     http://cubecobra.com/cube/api/cubeJSON/no-crea...
3           http://cubecobra.com/cube/api/cubeJSON/down
4     http://cubecobra.com/cube/api/cubeJSON/culticcube
5     http://cubecobra.com/cube/api/cubeJSON/pettyno...
6        http://cubecobra.com/cube/api/cubeJSON/dekkaru
7     http://cubecobra.com/cube/api/cubeJSON/tinylea...
8     http://cubecobra.com/cube/api/cubeJSON/emma-pe...
9            http://cubecobra.com/cube/api/cubeJSON/obf
10        http://cubecobra.com/cube/api/cubeJSON/devoid
11    http://cubecobra.com/cube/api/cubeJSON/extrava...
12    http://cubecobra.com/cube/api/cubeJSON/creativ...
13          http://cubecobra.com/cube/api/cubeJSON/data
14    http://cubecobra.com/cube/api/cubeJSON/pulpnou...
15    http://cubecobra.com/cube/api/cubeJSON/LiveThe...
16    http://cubecobra.com/cube/api/cubeJSON/amonkar...
17    http://cubecobra.com/cube/api/cubeJSON/Mod

In [6]:
# Start with an Index of 0
index = 0

# Start with an Empty List, the most functional of Data Types
df_list = []

for row in cube_slice.iterrows():
        if rq.get(cube_slice['URL'][index]).ok == True:
            df = pd.json_normalize(rq.get(cube_slice['URL'][index]).json().get('cards').get('mainboard'), sep='_')
            df['ID'] = cube_slice['ID'][index]
            df['Cube Name'] = cube_slice['Cube Name'][index]
            df['Cubecon Type'] = 'Pre-Accepted'
            df_list.append(df)
            index+=1
        else:
            print('Error Encountered At Index:', index)
            index+=1

In [7]:
pre_accepted_db = pd.concat(df_list)
pre_accepted_db.groupby(['Cube Name']).count()

,cmc,colorCategory,finish,isUnlimited,rarity,status,tags,type_line,colors,cardID,...,details_colorcategory,details_tokens,details_loyalty,ID,Cubecon Type,details_image_flip,imgUrl,imgBackUrl,notes,markedForDelete
Cube Name,,,,,,,,,,,,,,,,,,,,,
Amonkar Desert,387,0,387,387,11,480,387,387,397,480,...,480,93,1,480,480,14,0,0,0,6
Commander extravaganza!,429,6,434,429,0,500,429,429,429,500,...,500,89,7,500,500,6,0,0,0,75
Creatureless Cube,220,0,282,220,179,450,270,220,185,450,...,450,114,14,450,450,22,0,0,179,311
Data Generated Vintage Cube,293,16,353,293,0,360,294,293,295,360,...,360,50,23,360,360,3,22,0,21,130
Dekkaru Cube,480,0,479,479,0,517,479,479,479,517,...,517,71,9,517,517,0,150,0,26,28
Derek’s Cube,267,26,471,256,0,480,257,260,268,480,...,480,82,18,480,480,13,0,0,0,48
Dragons of Winter's Night,348,0,356,348,0,432,351,348,348,432,...,432,34,0,432,432,15,2,0,4,30
Eleusis,450,3,450,450,0,450,450,450,450,450,...,450,81,31,450,450,19,0,0,0,0
Emma Partlow's Peasant Cube,420,0,420,420,0,450,420,420,420,450,...,450,68,4,450,450,12,0,0,0,0


In [8]:
# Initial Fields as of 6/3/23
extract = pre_accepted_db.loc[:,[
      'index'
    , 'Cube Name'
    , 'Cubecon Type'
    , 'cardID'
    , 'addedTmsp'
    , 'details_name'
    , 'details_full_name'
    , 'details_artist'
    , 'details_rarity'
    , 'details_color_identity'
    , 'details_colors'
    , 'details_set'
    , 'details_released_at'
    , 'details_cmc'
    , 'details_parsed_cost'
    , 'details_type'
    , 'details_elo'
    , 'details_popularity'
    , 'details_cubeCount'
    , 'details_loyalty'
    , 'details_power'
    , 'details_toughness'
]]

In [9]:
extract.to_csv('../Data Files/pre_accepted_cubes_extract.csv', encoding='utf-8')

In [35]:
# Beneath Here Tests the Data

test_data = cube_slice.loc[0:3]

In [14]:
# request_1 = rq.get(test_data['URL'][0]).json().get('cards').get('mainboard')
# Create a DataFrame that's a Normalized JSON View of a Request from the URL Parameter of test_data's [0]th Value
# Which is turned into JSON, and give me the 'Cards' Subset, and the 'Mainboard'
# Also Separate the Normalized fields with '_'
df_1 = pd.json_normalize(rq.get(test_data['URL'][0]).json().get('cards').get('mainboard'), sep='_')
df_1['ID'] = test_data['ID'][0]
df_1['Cube Name'] = test_data['Cube Name'][0]
df_1['Cubecon Type'] = 'Pre-Accepted'
df_1

NameError: name 'test_data' is not defined